In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

import os

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
TARGET_SIZE = (100, 100)

## Data Augmentation 

In [4]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.2, 1.5]
)

### Generator of train images

In [5]:
data = os.path.join(os.getcwd(), 'dataset/split/train')

train_generator = datagen.flow_from_directory(   
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(train_generator)
print("Number of batches: ", num_batches)

Found 210 images belonging to 3 classes.
Number of batches:  7


### Generator of test images

In [6]:
data = os.path.join(os.getcwd(), 'dataset/split/test')

test_generator = datagen.flow_from_directory(
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(test_generator)
print("Number of batches: ", num_batches)

Found 60 images belonging to 3 classes.
Number of batches:  2


Son 100 imagenes, de 100x100, en los 3 canales RGB.

## Modelo de Datos

In [7]:
base_filters = 32 # 12 - 42
w_regularizer = 1e-1 # -1 -> -4

num_classes = len(train_generator.class_indices)

In [8]:
model = tf.keras.Sequential()

# First layer
model.add(Conv2D(filters=base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu', input_shape=(100,100,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Second layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3)) #0.2-0.6

# Third layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Fourth layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Fifth layer
model.add(Conv2D(filters=3*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Sixth layer
model.add(Conv2D(filters=3*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Flatten the output
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      320       
                                                                 
 activation (Activation)     (None, 100, 100, 32)      0         
                                                                 
 batch_normalization (BatchN  (None, 100, 100, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 64)      18496     
                                                                 
 activation_1 (Activation)   (None, 100, 100, 64)      0         
                                                                 
 batch_normalization_1 (Batc  (None, 100, 100, 64)     256       
 hNormalization)                                        

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [10]:
hist = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=100, verbose=1)

Epoch 1/1000
7/7 [==============================] - 4s 93ms/step - loss: 45.5044 - accuracy: 0.3429
Epoch 2/1000
7/7 [==============================] - 0s 33ms/step - loss: 37.1616 - accuracy: 0.3095
Epoch 3/1000
7/7 [==============================] - 0s 33ms/step - loss: 34.3183 - accuracy: 0.3714
Epoch 4/1000
7/7 [==============================] - 0s 33ms/step - loss: 32.6859 - accuracy: 0.3429
Epoch 5/1000
7/7 [==============================] - 0s 33ms/step - loss: 30.3792 - accuracy: 0.3476
Epoch 6/1000
7/7 [==============================] - 0s 33ms/step - loss: 26.7454 - accuracy: 0.3238
Epoch 7/1000
7/7 [==============================] - 0s 34ms/step - loss: 24.8660 - accuracy: 0.3857
Epoch 8/1000
7/7 [==============================] - 0s 33ms/step - loss: 22.4629 - accuracy: 0.3667
Epoch 9/1000
7/7 [==============================] - 0s 33ms/step - loss: 19.6100 - accuracy: 0.4476
Epoch 10/1000
7/7 [==============================] - 0s 34ms/step - loss: 18.8075 - accuracy: 0.3095

In [11]:
# Assuming test_images and test_labels are your testing data and labels
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

2/2 [==============================] - 0s 236ms/step - loss: 1.0751 - accuracy: 0.5500

Test accuracy: 0.550000011920929

Test loss: 1.0751315355300903


## Save model 

In [12]:
# Save the model
model.save('out/cnn2.keras')